In [643]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted
from sklearn.pipeline import Pipeline
from datetime import datetime as dt
import re

# Step 1: Standardize null values and white space(s) strings to np.nan;
#              Standardize potential boolean values to be True or False;
#              Convert all values to string except NaN

In [644]:
class standardizer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.null_patterns = re.compile(r'^N[./]*A[./]*[NT]?[./]*$|^none[.]?$|^null[.]?$', re.IGNORECASE)
        self.empty_patterns = re.compile(r'^\s*$', re.IGNORECASE)
        self.true_patterns = re.compile(r'^True$', re.IGNORECASE)
        self.false_patterns = re.compile(r'^False$', re.IGNORECASE)
    def fit(self, X, y = None):
        columns = []
        for col in X.columns:
            try:
                X[col].apply(lambda x: np.nan if bool(self.null_patterns.search(str(x))) else x)
                X[col].apply(lambda x: np.nan if bool(self.empty_patterns.search(str(x))) else x)
                X[col].apply(lambda x: True if bool(self.true_patterns.search(str(x))) else x)
                X[col].apply(lambda x: False if bool(self.false_patterns.search(str(x))) else x)
                X[col].apply(lambda x: x if pd.isnull(x) else str(x))      
                columns.append(col)
            except:
                pass
        self.columns = columns
        return self
    
    def transform(self, X, y=None):
        check_is_fitted(self,['columns'])
        X_t = X.copy()
        for col in self.columns:
            X_t[col] = X_t[col].apply(lambda x: np.nan if bool(self.null_patterns.search(str(x))) else x)
            X_t[col] = X_t[col].apply(lambda x: np.nan if bool(self.empty_patterns.search(str(x))) else x)
            X_t[col] = X_t[col].apply(lambda x: True if bool(self.true_patterns.search(str(x))) else x)
            X_t[col] = X_t[col].apply(lambda x: False if bool(self.false_patterns.search(str(x))) else x)
            X_t[col] = X_t[col].apply(lambda x: x if pd.isnull(x) else str(x))    
        return X_t

# Step 2: Transform numerical columns

In [645]:
class numerical_transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        columns = []
        for col in X.columns:
            if '_t' not in col:
                try:
                    X[col].apply(lambda x: '' if pd.isnull(x) else float(x))
                    columns.append(col)
                except:
                    pass
        self.columns = columns
        return self
    
    def transform(self, X, y = None):
        check_is_fitted(self,['columns'])
        X_t = X.copy()
        for col in self.columns:
            X_t[col] = X_t[col].apply(lambda x: '' if pd.isnull(x) else float(x))
            X_t = X_t.rename(columns={col : col + '_t'})
        return X_t

# Step 3: Transform date columns

In [646]:
class date_transformer(BaseEstimator, TransformerMixin):       
    def _convert_to_date(self, x):
        try:
            return dt.strptime(x, '%Y-%m-%d').date()
        except:
            try:
                return dt.strptime(x, '%Y%m%d').date()
            except ValueError:
                pass
        raise ValueError(f"Cannot convert {x} to a date with the given formats.")

    def fit(self, X, y = None):
        columns = []
        for col in X.columns:
            if '_t' not in col:
                try:
                    X[col].apply(lambda x: ' ' if pd.isnull(x) else self._convert_to_date(x))
                    columns.append(col)
                except ValueError:
                    pass
        self.columns = columns
        return self
    
    def transform(self, X, y = None):
        check_is_fitted(self,['columns'])
        X_t = X.copy()
        for col in self.columns:
            X_t[col] = X_t[col].apply(lambda x: ' ' if pd.isnull(x) else self._convert_to_date(x))
            X_t = X_t.rename(columns={col : col + '_t'})
        return X_t

# Step 4: Preprocess string columns

In [647]:
class string_transformer(BaseEstimator, TransformerMixin):
    def _convert_to_string(self, x):
        if x in ['True', 'False']:
            raise ValueError("The value cannot be 'True' or 'False'.")
        else:
            return str(x)
    
    def fit(self, X, y = None):
        columns = []
        for col in X.columns:
            if '_t' not in col:
                try:
                    X[col].apply(lambda x: ' ' if pd.isnull(x) else self._convert_to_string(x))
                    columns.append(col)
                except ValueError:
                    pass
        self.columns = columns
        return self
    
    def transform(self, X, y = None):
        check_is_fitted(self,['columns'])
        X_t = X.copy()
        for col in self.columns:
            X_t[col] = X_t[col].apply(lambda x: ' ' if pd.isnull(x) else self._convert_to_string(x))
            X_t = X_t.rename(columns={col : col + '_t'})
        return X_t

# Step 4: Preprocess boolean columns

In [648]:
class boolean_transformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y = None):
        columns = []
        for col in X.columns:
            if '_t' not in col:
                try:
                    X[col].apply(lambda x: '' if pd.isnull(x) else bool(x))
                    columns.append(col)
                except:
                    pass
        self.columns = columns
        return self
    
    def transform(self, X, y = None):
        check_is_fitted(self,['columns'])
        X_t = X.copy()
        for col in self.columns:
            X_t[col] = X_t[col].apply(lambda x: '' if pd.isnull(x) else bool(x))
#             X_t = X_t.rename(columns={col : col + '_t'})
        X_t = X_t.rename(columns=lambda x: x[:-2] if x.endswith('_t') else x)
        return X_t

# Validation

In [676]:
df = pd.DataFrame({
    'Numerical':['123',' ','','NA','N.A.','None','20',2.5,3.8,np.nan],
    'Boolean':['True',' ','','NA','N.A.','None',True,False,False,np.nan],
    'Character':['abc',' ','','NA','N.A.','None','cde',1234,'234',12],
    'Date':['2023-06-28',' ','','NA','N.A.','None',20230629,20230630,'20230630',np.nan]})

In [650]:
s = standardizer()

In [651]:
df_t = s.fit_transform(df)

In [652]:
df_t

,Numerical,Boolean,Character,Date
0,123,True,abc,2023-06-28
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN
6,20,True,cde,20230629
7,2.5,False,1234,20230630
8,3.8,False,234,20230630
9,NaN,NaN,12,NaN


In [653]:
n = numerical_transformer()

In [654]:
df_t = n.fit_transform(df_t)

In [655]:
df_t

,Numerical_t,Boolean,Character,Date
0,123.0,True,abc,2023-06-28
1,,NaN,NaN,NaN
2,,NaN,NaN,NaN
3,,NaN,NaN,NaN
4,,NaN,NaN,NaN
5,,NaN,NaN,NaN
6,20.0,True,cde,20230629
7,2.5,False,1234,20230630
8,3.8,False,234,20230630
9,,NaN,12,NaN


In [656]:
d = date_transformer()

In [657]:
df_t = d.fit_transform(df_t)

In [658]:
df_t

,Numerical_t,Boolean,Character,Date_t
0,123.0,True,abc,2023-06-28
1,,NaN,NaN,
2,,NaN,NaN,
3,,NaN,NaN,
4,,NaN,NaN,
5,,NaN,NaN,
6,20.0,True,cde,2023-06-29
7,2.5,False,1234,2023-06-30
8,3.8,False,234,2023-06-30
9,,NaN,12,


In [659]:
st = string_transformer()

In [660]:
df_t = st.fit_transform(df_t)

In [661]:
df_t

,Numerical_t,Boolean,Character_t,Date_t
0,123.0,True,abc,2023-06-28
1,,NaN,,
2,,NaN,,
3,,NaN,,
4,,NaN,,
5,,NaN,,
6,20.0,True,cde,2023-06-29
7,2.5,False,1234,2023-06-30
8,3.8,False,234,2023-06-30
9,,NaN,12,


In [662]:
b = boolean_transformer()

In [663]:
df_t = b.fit_transform(df_t)

In [664]:
df_t

,Numerical,Boolean,Character,Date
0,123.0,True,abc,2023-06-28
1,,,,
2,,,,
3,,,,
4,,,,
5,,,,
6,20.0,True,cde,2023-06-29
7,2.5,True,1234,2023-06-30
8,3.8,True,234,2023-06-30
9,,,12,


In [665]:
df_t.applymap(type)

,Numerical,Boolean,Character,Date
0,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
5,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
6,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
7,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
8,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
9,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


# Test with applying pipeline

In [666]:
df = pd.DataFrame({
    'Numerical':['123',' ','','NA','N.A.','None','20',2.5,3.8,np.nan],
    'Boolean':['True',' ','','NA','N.A.','None',True,False,False,np.nan],
    'Character':['abc',' ','','NA','N.A.','None','cde',1234,'234',12],
    'Date':['2023-06-28',' ','','NA','N.A.','None',20230629,20230630,'20230630',np.nan]})

In [680]:
# The transformed DataFrame should look at the following:
# df_t = pd.DataFrame({
#     'Numerical':[123.0,'','','','','',20.0,2.5,3.8,''],
#     'Boolean':[True,'','','','','',True,False,False,''],
#     'Character':['abc',' ',' ',' ',' ',' ','cde','1234','234','12'],
#     'Date':['2023-06-28',' ',' ',' ',' ',' ','2023-06-29','2023-06-30','2023-06-30',' ']})

In [668]:
pipeline = Pipeline(steps=[
    ('standardize', standardizer()),
    ('numerical', numerical_transformer()),
    ('date', date_transformer()),
    ('string', string_transformer()),
    ('boolean', boolean_transformer())
])

In [669]:
df_t = pipeline.fit_transform(df)

In [670]:
df_t

,Numerical,Boolean,Character,Date
0,123.0,True,abc,2023-06-28
1,,,,
2,,,,
3,,,,
4,,,,
5,,,,
6,20.0,True,cde,2023-06-29
7,2.5,True,1234,2023-06-30
8,3.8,True,234,2023-06-30
9,,,12,


In [671]:
df_t.applymap(type)

,Numerical,Boolean,Character,Date
0,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
5,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
6,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
7,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
8,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'datetime.date'>
9,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [672]:
df_t.to_csv('df_t.csv', index=False)

In [673]:
df_t = pd.read_csv('df_t.csv')

In [674]:
df_t.applymap(type)

,Numerical,Boolean,Character,Date
0,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'str'>
1,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>
2,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>
3,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>
4,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>
5,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>
6,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'str'>
7,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'str'>
8,<class 'float'>,<class 'bool'>,<class 'str'>,<class 'str'>
9,<class 'float'>,<class 'float'>,<class 'str'>,<class 'str'>


In [675]:
df_t

,Numerical,Boolean,Character,Date
0,123.0,True,abc,2023-06-28
1,NaN,NaN,,
2,NaN,NaN,,
3,NaN,NaN,,
4,NaN,NaN,,
5,NaN,NaN,,
6,20.0,True,cde,2023-06-29
7,2.5,True,1234,2023-06-30
8,3.8,True,234,2023-06-30
9,NaN,NaN,12,
